In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz

from pprint import pprint

import util
import lines
import classification

In [2]:
test_files_dir = ".././test_files/"
test_files = util.list_files(test_files_dir)
test_files

['.././test_files/LS_index_1944.pdf',
 '.././test_files/LS_index_1986_split.pdf',
 '.././test_files/Argentinia_LS_index_1934_test_I_DE.pdf',
 '.././test_files/LS_index_1976.pdf',
 '.././test_files/LS_index_1920.pdf',
 '.././test_files/LS_index_1986.pdf',
 '.././test_files/LS_index_1937.pdf',
 '.././test_files/LS_index_1920_split.pdf',
 '.././test_files/LS_index_1944_split.pdf']

In [3]:
p_start = 1
pages, dicts = util.read_pdf(test_files[-4], p_start)

l_df = lines.make_lines_df(dicts)
l_df = lines.merge_close_lines(l_df)
l_df = lines.remove_useless_lines(l_df)

display(l_df)

Reading pdf from .././test_files/LS_index_1986.pdf
...
Finished reading 89 page(s)


,line_text,spans,x0,y0,x1,y1,page
0,International,[International ],40.32,40.85,86.17,51.93,1
1,Labour Office,[Labour Office ],40.56,49.73,90.48,60.81,1
2,Legislative Series,[Legislative Series ],306.48,41.09,369.71,52.17,1
3,1987—List No. 3,[1987—List No. 3 ],307.68,49.49,369.88,60.57,1
4,LIST OF RECENT LEGISLATION,[LIST OF RECENT LEGISLATION ],105.60,69.11,305.78,86.45,1
...,...,...,...,...,...,...,...
4655,(195) 89 1987—List No. 3,"[(195) , 89 , 1987—List No. 3 ]",43.68,45.00,373.53,57.01,89
4656,Spain—Uruguay,[Spain—Uruguay ],43.92,70.04,103.60,82.51,89
4657,19/11/1986 Additional international technical ...,[19/11/1986 Additional international technical...,44.64,81.96,373.93,93.97,89
4658,matters between the Republic of Uruguay and th...,[matters between the Republic of Uruguay and t...,55.68,90.36,379.08,102.37,89


In [4]:
bins_x0, bins_x1, x0_n = classification.group_line_starts_ends(l_df)
display(x0_n, bins_x0, bins_x1)

2

,x0,lines,last_x0,count,page
3,"[40.32, 40.56, 39.36, 40.56, 39.6, 40.56, 40.3...","[0, 1, 6, 8, 9, 10, 12, 14, 33, 35, 38, 39, 53]",39.12,13,1
0,"[51.84, 52.08, 51.36, 51.36, 52.08, 51.36, 51....","[5, 7, 11, 13, 15, 16, 17, 18, 19, 20, 21, 22,...",51.84,38,1
1,"[75.36, 73.92, 73.92, 73.92, 73.44, 74.4, 73.9...","[54, 55, 56, 64, 70, 72, 79, 82, 83, 87, 90, 9...",73.92,14,2
0,"[85.92, 86.64, 85.92, 85.92, 86.16, 86.16, 86....","[57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 6...",86.16,36,2
1,"[36.48, 37.44, 37.44, 37.2, 37.44, 37.68, 37.4...","[104, 105, 106, 109, 113, 117, 118, 121, 128, ...",38.4,11,3
...,...,...,...,...,...
1,"[51.36, 51.6, 51.6, 51.6, 52.08, 51.6, 51.84, ...","[4557, 4558, 4561, 4562, 4563, 4564, 4565, 456...",52.8,40,87
1,"[73.44, 72.48, 72.96, 72.96, 72.24, 72.96, 72....","[4606, 4607, 4608, 4615, 4617, 4618, 4627, 462...",72.0,14,88
0,"[84.48, 84.72, 84.0, 84.0, 84.0, 84.96, 84.24,...","[4609, 4610, 4611, 4612, 4613, 4614, 4616, 461...",84.24,35,88
1,"[43.68, 43.92, 44.64]","[4655, 4656, 4657]",44.64,3,89


,x1,lines,last_x1,count,page
9,"[369.71, 369.88, 370.1, 369.7, 369.64, 370.18,...","[2, 3, 5, 7, 10, 12, 15, 16, 17, 18, 19, 20, 2...",369.37,39,1
16,"[404.51, 404.79, 404.85, 404.68, 404.74, 405.1...","[54, 56, 58, 59, 60, 61, 62, 64, 66, 67, 68, 7...",404.07,27,2
16,"[366.33, 367.58, 367.75, 367.42, 368.74, 368.0...","[104, 106, 108, 109, 111, 113, 115, 118, 119, ...",366.67,34,3
14,"[406.23, 405.93, 405.65, 405.68, 405.57, 405.4...","[158, 160, 161, 163, 164, 165, 166, 167, 168, ...",405.26,31,4
19,"[369.6, 367.58, 368.88, 368.03, 368.26, 367.93...","[210, 212, 215, 217, 218, 221, 222, 223, 224, ...",366.69,21,5
...,...,...,...,...,...
17,"[378.76, 379.15, 380.6, 380.1, 383.27]","[4463, 4466, 4470, 4475, 4490]",383.27,5,85
15,"[402.11, 403.08, 403.2, 403.78, 403.15, 402.52...","[4506, 4508, 4512, 4516, 4517, 4519, 4520, 452...",403.28,31,86
13,"[370.17, 370.09, 370.12, 372.96, 372.01, 370.5...","[4555, 4560, 4563, 4564, 4565, 4566, 4567, 456...",367.82,34,87
15,"[403.07, 403.49, 403.0, 403.49, 403.49, 405.08...","[4606, 4608, 4610, 4611, 4612, 4613, 4615, 461...",402.85,30,88


In [5]:
li_df = classification.assign_types(l_df, bins_x0, bins_x1, x0_n)
li_df.head(20)

,line_text,spans,x0,y0,x1,y1,page,x0_type,x1_type
0,International,[International ],40.32,40.85,86.17,51.93,1,0,0
1,Labour Office,[Labour Office ],40.56,49.73,90.48,60.81,1,0,0
2,Legislative Series,[Legislative Series ],306.48,41.09,369.71,52.17,1,-1,2
3,1987—List No. 3,[1987—List No. 3 ],307.68,49.49,369.88,60.57,1,-1,2
4,LIST OF RECENT LEGISLATION,[LIST OF RECENT LEGISLATION ],105.60,69.11,305.78,86.45,1,-1,1
5,This list indicates the more important enactme...,[This list indicates the more important enactm...,51.84,99.19,370.10,108.96,1,1,2
6,noted in the official gazettes recently receiv...,[noted in the official gazettes recently recei...,39.36,107.59,314.19,117.36,1,0,1
7,"[This cumulative list, arranged in chronologic...","[[This cumulative list, arranged in chronologi...",52.08,115.42,369.70,126.09,1,1,2
8,1986.],[1986.] ],40.56,123.82,62.32,134.49,1,0,0
9,ALGERIA,[ALGERIA ],39.60,148.13,78.70,159.21,1,0,0


In [6]:
cli_df = classification.correct_x0_types(li_df, bins_x0, bins_x1, x0_n)
ll_df = classification.assign_labels(cli_df, x0_n)
ll_df.to_csv("labeling_1986_full.csv")
ll_df

,line_text,spans,x0,y0,x1,y1,page,x0_type,x1_type,label
0,International,[International ],40.32,40.85,86.17,51.93,1,0,0,country
1,Labour Office,[Labour Office ],40.56,49.73,90.48,60.81,1,0,0,country
2,Legislative Series,[Legislative Series ],306.48,41.09,369.71,52.17,1,-1,2,other
3,1987—List No. 3,[1987—List No. 3 ],307.68,49.49,369.88,60.57,1,-1,2,other
4,LIST OF RECENT LEGISLATION,[LIST OF RECENT LEGISLATION ],105.60,69.11,305.78,86.45,1,-1,1,other
...,...,...,...,...,...,...,...,...,...,...
4655,(195) 89 1987—List No. 3,"[(195) , 89 , 1987—List No. 3 ]",43.68,45.00,373.53,57.01,89,-1,1,other
4656,Spain—Uruguay,[Spain—Uruguay ],43.92,70.04,103.60,82.51,89,-1,0,other
4657,19/11/1986 Additional international technical ...,[19/11/1986 Additional international technical...,44.64,81.96,373.93,93.97,89,-1,1,other
4658,matters between the Republic of Uruguay and th...,[matters between the Republic of Uruguay and t...,55.68,90.36,379.08,102.37,89,0,2,start


In [7]:
ll_df.loc[ll_df["page"]==64]

,line_text,spans,x0,y0,x1,y1,page,x0_type,x1_type,label
3319,1044—Indexes II — 64 —,"[1044—Indexes II, —, 64, —]",55.0,50.90,230.40,60.48,64,0,1,other
3320,"May 19 Act No. 226, to amend sections 1 and 4 ...","[May, 19, Act No. 226, to amend sections 1 and...",64.0,83.48,373.41,93.88,64,1,2,start
3321,"June, 1929, respecting insurance against• cert...","[June,, 1929,, respecting, insurance, against•...",111.0,92.23,371.46,102.03,64,2,2,middle
3322,(S.F. pp. 439-440) Swe. I (A),"[(S.F. pp. 439-440), Swe., I, (A)]",160.0,100.46,371.46,110.20,64,-1,2,other
3323,"May 19 Act No. 227, to amend section","[May 19, Act No. 227, to amend section]",64.0,114.00,234.15,124.10,64,1,1,other
3324,1,[1],241.0,117.88,243.40,122.68,64,-1,1,other
3325,"of Act No. 235 of 17th June,","[of Act No. 235 of 17th June,]",250.0,113.53,371.30,124.13,64,-1,2,other
3326,"1916, respecting insurance against industrial ...","[1916,, respecting, insurance, against, indust...",111.0,121.93,371.25,132.53,64,2,2,middle
3327,p. 441) :,"[p. 441), :]",111.0,131.68,170.19,140.88,64,2,0,end
3328,Swe. I (B),"[Swe. I, (B)]",328.0,130.80,370.93,140.50,64,-1,2,other


In [55]:
classification.improve_classification(ll_df)

,line_text,spans,x0,y0,x1,y1,page,x0_type,x1_type,label,new_label,date
0,1.—LIST OF LAWS AND ORDERS OF 1944 NOT PUBLISH...,[1.—LIST OF LAWS AND ORDERS OF 1944 NOT PUBLIS...,35.0,94.69,342.19,104.79,3,1,2,start,other,
1,"LEGISLATIVE SERIES FOR THAT YEAR WHICH AMEND, RE","[LEGISLATIVE SERIES FOR THAT YEAR WHICH AMEND,...",48.0,104.29,336.14,114.79,3,-1,1,other,other,
2,PEAL OR SUPERSEDE LA\VS AND ORDERS PUBLISHED PRE,[PEAL OR SUPERSEDE LA\VS AND ORDERS PUBLISHED ...,49.0,114.39,338.64,124.89,3,-1,2,other,other,
3,VIOUSLY IN THE LEGISLATIVE SERIES,[VIOUSLY IN THE LEGISLATIVE SERIES],48.0,124.46,236.91,134.86,3,-1,1,other,other,
4,ARGENTINE REPUBLIC,[ARGENTINE REPUBLIC],26.0,147.26,151.34,158.86,3,0,0,country,country,
...,...,...,...,...,...,...,...,...,...,...,...,...
4320,Apr. 28 L.D. respecting the organisation of la...,"[Apr. 28, L.D., respecting the organisation of...",62.0,528.92,369.30,539.32,82,1,2,start,start,Apr. 28
4321,agreements; labour courts]. No. 311. (Zb. pp. ...,"[agreements; labour courts]., No. 311., (Zb. p...",109.0,537.12,337.55,547.49,82,2,1,end,end,
4322,G.O. = Gacefa Oficial de los Unmdos de Venezuela.,"[G.O. =, Gacefa, Oficial de los, Unmdos de Ven...",63.0,552.82,296.01,563.62,82,1,1,other,other,
4323,Zb. = Zbornile zakona i naredaba.,"[Zb. =, Zbornile, zakona, i, naredaba.]",62.0,562.45,195.35,573.75,82,1,0,other,other,


## Probleme
- Dokumente mit 2 Spalten
- Dokumente mit Länder-Überschrift über ganze Breite der Seite
- Erkennen, ab welcher Seite die eigentlichen Indexes beginnen
- Unnötigen Anfangstext auf der ersten Seite rausfiltern

## Fragen
